In [2]:
%load_ext autoreload
%autoreload 2
import sys
import platform
from pathlib import Path
from os import environ
if platform.system() == "Darwin": # Nat laptop
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/Projects_sandbox'])
    plot_dir = Path("/Users/nkinsky/University of Michigan Dropbox/Nathaniel Kinsky/Manuscripts/Psilocybin/plots")
else:
    if environ["HOSTNAME"] == "lnx00004": # Nat Linux computer
        sys.path.extend(['/data/GitHub/NeuroPy'])
        sys.path.extend(['/data/GitHub/Projects_sandbox'])
        plot_dir = Path("/home/nkinsky/Dropbox (University of Michigan)/Manuscripts/Psilocybin/plots")
        
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy.utils.mathutil import min_max_scaler
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from copy import deepcopy
from tqdm import tqdm

from neuropy import plotting
from neuropy.analyses.placefields import Pf1D, Pf1Dsplit
from neuropy.analyses.oscillations import detect_theta_epochs
from neuropy.core.position import Position
from neuropy.core.epoch import Epoch
from neuropy.plotting.ratemaps import plot_ratemap
from neuropy.plotting.figure import Fig
from neuropy.utils.misc import flatten
from neuropy.io import BinarysignalIO

from Psilocybin.subjects import get_psi_dir

# These are crucial parameters - otherwise all text gets output with each letter in a word or heading as a separate unit
# instead of a text box
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# Specify Arial as font type - also crucial
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Specify plotting parameters

In [3]:
animals = ["Finn", "Rey", "Rose", "Finn2"]
sessions = ["Saline1", "Psilocybin", "Saline2"]
sessions_full = ["Saline 1", "Psilocybin", "Saline 2"]
rasterize_scatter = True # This is crucial, default (False) will produce unweildy plots

In [4]:
ripple_thresh = 4 # 2.5 or 4

# Cut down Finn2 saline to 1hr?
chop_finn2_saline = False  # True = only use 1st hour of Finn2 saline, False = use all
finn2_append = "_1hrsalineonly" if chop_finn2_saline else ""

# ... OR only use 1hr Psilocybin for all
limit_to_1st_hr = False
chop_all_append = "_allsessions1hr" if limit_to_1st_hr else ""
finn2_append = "" if chop_all_append else finn2_append

## Load in data

In [5]:
df_grp = []
for ida, animal in enumerate(animals):
    file_use = get_psi_dir(animal, "Saline1").parent / "aggdata" / f"{animal.lower()}_rpl_features_thresh{'_'.join(str(ripple_thresh).split('.'))}{chop_all_append}.csv"
    df_animal = pd.read_csv(file_use, index_col=0)
    df_animal["Animal_name"] = animal
    df_animal["Animal"] = ida + 1
    df_grp.append(df_animal)

df_grp = pd.concat(df_grp, axis=0, ignore_index=True)
df_grp["session"] = pd.Categorical(df_grp["session"], categories=["Saline 1", "Psilocybin", "Saline 2"])

df_grp
    

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,Animal_name,Animal
0,0.9184,0.9992,0.9712,28540.330,197.917961,0.0808,168.181818,NaN,13.364659,Saline 1,Finn,1
1,69.9744,70.0368,70.0056,43174.117,319.979066,0.0624,221.212121,NaN,6.180823,Saline 1,Finn,1
2,70.4328,70.5104,70.4736,25572.940,223.388474,0.0776,166.666667,NaN,3.513697,Saline 1,Finn,1
3,89.4656,89.5320,89.5032,35710.438,278.840761,0.0664,171.212121,NaN,6.030679,Saline 1,Finn,1
4,120.9960,121.0720,121.0464,28893.555,223.479148,0.0760,187.878788,NaN,6.569296,Saline 1,Finn,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14151,11503.6800,11503.7568,11503.7296,25091.900,182.681418,0.0768,162.121212,NaN,8.826762,Saline 2,Finn2,4
14152,11506.7480,11506.9304,11506.8040,29277.814,181.751974,0.1824,125.757576,NaN,8.691295,Saline 2,Finn2,4
14153,11509.3720,11509.4984,11509.4296,38501.550,302.989666,0.1264,186.363636,NaN,9.745170,Saline 2,Finn2,4
14154,11520.1152,11520.2144,11520.1496,49521.440,321.642974,0.0992,163.636364,NaN,20.808641,Saline 2,Finn2,4


In [6]:
df_grp.groupby(["Animal", "session"]).count()

start  stop  peak_time  peak_power  peak_power_abs  \
Animal session                                                          
1      Saline 1      484   484        484         484             484   
       Psilocybin   1739  1739       1739        1739            1739   
       Saline 2      372   372        372         372             372   
2      Saline 1      418   418        418         418             418   
       Psilocybin   1228  1228       1228        1228            1228   
       Saline 2     1464  1464       1464        1464            1464   
3      Saline 1      600   600        600         600             600   
       Psilocybin    656   656        656         656             656   
       Saline 2      673   673        673         673             673   
4      Saline 1     1927  1927       1927        1927            1927   
       Psilocybin   3349  3349       3349        3349            3349   
       Saline 2     1246  1246       1246        1246            1246   

                   duration  peak_frequency_bp  label  sharp_wave_amplitude  \
Animal session                                                                
1      Saline 1         484                484      0                   484   
       Psilocybin      1739               1739      0                  1739   
       Saline 2         372                372      0                   372   
2      Saline 1         418                418      0                   418   
       Psilocybin      1228               1228      0                  1228   
       Saline 2        1464               1464      0                  1464   
3      Saline 1         600                600      0                   600   
       Psilocybin       656                656      0                   656   
       Saline 2         673                673      0                   673   
4      Saline 1        1927               1927      0                  1927   
       Psilocybin      3349               3349      0                  3349   
       Saline 2        1246               1246      0                  1246   

                   Animal_name  
Animal session                  
1      Saline 1            484  
       Psilocybin         1739  
       Saline 2            372  
2      Saline 1            418  
       Psilocybin         1228  
       Saline 2           1464  
3      Saline 1            600  
       Psilocybin          656  
       Saline 2            673  
4      Saline 1           1927  
       Psilocybin         3349  
       Saline 2           1246

In [7]:
# Adjust injection times to 0 = injection time
for animal in animals:
    for session, session_full in zip(sessions, sessions_full):
        inj_file = get_psi_dir(animal, session)
        inj_epochs = Epoch(epochs=None, file=sorted(inj_file.glob("*.injection.npy"))[0])
        inj_time = inj_epochs["POST"].starts[0]
        sesh_bool = (df_grp.Animal_name == animal) & (df_grp.session == session_full)
        df_grp.loc[sesh_bool, "start"] = df_grp.loc[sesh_bool, "start"] - inj_time
        df_grp.loc[sesh_bool, "stop"] = df_grp.loc[sesh_bool, "stop"] - inj_time
        df_grp.loc[sesh_bool, "peak_time"] = df_grp.loc[sesh_bool, "peak_time"] - inj_time

In [8]:
df_mean = df_grp.groupby(["Animal", "session"]).mean(numeric_only=True).reset_index()
# df_mean["session"] = pd.Categorical(df_mean["session"], categories=["Saline 1", "Psilocybin", "Saline 2"])
df_mean

,Animal,session,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude
0,1,Saline 1,1819.107369,1819.222430,1819.164260,41008.830112,289.762422,0.115061,168.964438,NaN,6.388298
1,1,Psilocybin,4817.876276,4817.990815,4817.935313,44201.338111,288.340668,0.114539,160.939760,NaN,5.688975
2,1,Saline 2,1158.382781,1158.488770,1158.437000,31041.087282,218.454593,0.105989,169.660313,NaN,6.732229
3,2,Saline 1,1289.023148,1289.127409,1289.076052,25770.904792,189.061133,0.104260,170.157315,NaN,8.091481
4,2,Psilocybin,2728.056944,2728.191671,2728.124632,25652.341664,173.410084,0.134727,161.701708,NaN,8.381922
5,2,Saline 2,2235.835499,2235.937142,2235.888654,44304.921286,319.780045,0.101642,174.030262,NaN,7.765844
6,3,Saline 1,764.646761,764.756966,764.702372,31125.993390,230.691436,0.110205,173.027778,NaN,7.854424
7,3,Psilocybin,786.362774,786.490434,786.425080,25155.196785,180.437229,0.127660,166.119272,NaN,5.397937
8,3,Saline 2,1074.792766,1074.906081,1074.850148,24094.556942,177.963076,0.113315,170.158494,NaN,6.376375
9,4,Saline 1,4448.623710,4448.742484,4448.682899,28441.120150,201.782237,0.118773,166.625780,NaN,5.599844


#### Double check vs Ilknur data

In [9]:
df_clip = df_grp[(df_grp.start < 3600) & (df_grp.start > 0)]
df_clip

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,Animal_name,Animal
0,0.91840,0.99920,0.97120,28540.330,197.917961,0.0808,168.181818,NaN,13.364659,Saline 1,Finn,1
1,69.97440,70.03680,70.00560,43174.117,319.979066,0.0624,221.212121,NaN,6.180823,Saline 1,Finn,1
2,70.43280,70.51040,70.47360,25572.940,223.388474,0.0776,166.666667,NaN,3.513697,Saline 1,Finn,1
3,89.46560,89.53200,89.50320,35710.438,278.840761,0.0664,171.212121,NaN,6.030679,Saline 1,Finn,1
4,120.99600,121.07200,121.04640,28893.555,223.479148,0.0760,187.878788,NaN,6.569296,Saline 1,Finn,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13517,3557.13642,3557.19962,3557.17242,23479.060,169.605681,0.0632,163.636364,NaN,6.191336,Saline 2,Finn2,4
13518,3569.69402,3569.76282,3569.72922,35307.703,279.906008,0.0688,178.787879,NaN,5.965886,Saline 2,Finn2,4
13519,3580.26362,3580.33882,3580.30362,31771.880,213.669109,0.0752,154.545455,NaN,6.475258,Saline 2,Finn2,4
13520,3581.41082,3581.58842,3581.54122,29473.210,234.280787,0.1776,172.727273,NaN,12.476437,Saline 2,Finn2,4


In [11]:
df_count = df_clip.loc[:, ["session", "Animal_name", "start"]].groupby(["Animal_name", "session"]).count()
df_count.rename(columns={"start": "rpl_count"})

rpl_count
Animal_name session              
Finn        Saline 1          484
            Psilocybin        584
            Saline 2          372
Finn2       Saline 1          738
            Psilocybin        782
            Saline 2          496
Rey         Saline 1          418
            Psilocybin        467
            Saline 2         1082
Rose        Saline 1          476
            Psilocybin        499
            Saline 2          499

In [14]:
# Compare to Ilknur data after bugfix
ilknur_new_df = pd.read_csv("/data3/Psilocybin/Recording_Rats/ripple_features_group_from_final_plots.csv", index_col=0)
ilknur_new_df["session"] = pd.Categorical(ilknur_new_df["session"], ["Saline 1", "Psilocybin", "Saline 2"])
ilknur_new_df

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,animal,animal_label
0,20.20880,20.31600,20.27200,24817.521,211.327125,0.1072,183.333333,NaN,10.596831,Saline 1,rey,Animal 1
1,33.02320,33.10640,33.07120,18607.932,138.054542,0.0832,153.030303,NaN,36.626322,Saline 1,rey,Animal 1
2,33.14960,33.40560,33.30400,39154.790,281.620276,0.2560,163.636364,NaN,12.076338,Saline 1,rey,Animal 1
3,35.19600,35.24720,35.22000,21863.021,156.924436,0.0512,177.272727,NaN,7.478112,Saline 1,rey,Animal 1
4,44.26720,44.37200,44.34160,25581.988,169.747485,0.1048,115.151515,NaN,6.170822,Saline 1,rey,Animal 1
...,...,...,...,...,...,...,...,...,...,...,...,...
6731,3557.13642,3557.19962,3557.17242,23479.060,169.605681,0.0632,163.636364,NaN,6.141340,Saline 2,finn2,Animal 4
6732,3569.69402,3569.76282,3569.72922,35307.703,279.906008,0.0688,178.787879,NaN,6.294665,Saline 2,finn2,Animal 4
6733,3580.26362,3580.33882,3580.30362,31771.880,213.669109,0.0752,154.545455,NaN,6.274445,Saline 2,finn2,Animal 4
6734,3581.41082,3581.58842,3581.54122,29473.210,234.280787,0.1776,172.727273,NaN,12.184149,Saline 2,finn2,Animal 4


In [19]:
df_clip[(df_clip.Animal_name == "Rey") & (df_clip.session == "Saline 2")]

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,Animal_name,Animal
4241,17.0472,17.1976,17.0976,31091.553,247.127019,0.1504,212.121212,NaN,9.505339,Saline 2,Rey,2
4242,40.3408,40.4536,40.4088,44794.285,270.459223,0.1128,154.545455,NaN,8.808732,Saline 2,Rey,2
4243,42.4144,42.4864,42.4544,35997.598,267.091076,0.0720,169.696970,NaN,6.360695,Saline 2,Rey,2
4244,47.8592,48.0000,47.9304,31431.363,254.598665,0.1408,181.818182,NaN,8.940692,Saline 2,Rey,2
4245,48.1760,48.2776,48.2328,44729.703,353.745384,0.1016,171.212121,NaN,8.159270,Saline 2,Rey,2
...,...,...,...,...,...,...,...,...,...,...,...,...
5318,3501.3632,3501.4560,3501.3984,32680.570,251.626760,0.0928,139.393939,NaN,8.015137,Saline 2,Rey,2
5319,3503.5016,3503.6192,3503.5728,53479.812,417.950594,0.1176,180.303030,NaN,4.316168,Saline 2,Rey,2
5320,3503.7216,3503.7992,3503.7544,30071.207,256.521518,0.0776,174.242424,NaN,5.732398,Saline 2,Rey,2
5321,3507.5952,3507.7496,3507.7160,50313.676,373.281839,0.1544,180.303030,NaN,6.146879,Saline 2,Rey,2


In [16]:
ilknur_new_df[(ilknur_new_df.animal == "rey") & (ilknur_new_df.session == "Saline 2")]

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,animal,animal_label
885,9.1520,9.3224,9.2096,178223.830,778.297412,0.1704,119.696970,NaN,44.315761,Saline 2,rey,Animal 1
886,17.0528,17.1648,17.0952,21208.220,168.016621,0.1120,219.696970,NaN,9.878317,Saline 2,rey,Animal 1
887,18.3224,18.3912,18.3432,20371.264,150.957396,0.0688,193.939394,NaN,6.068069,Saline 2,rey,Animal 1
888,40.3280,40.4504,40.3920,27379.646,152.023652,0.1224,148.484848,NaN,7.207715,Saline 2,rey,Animal 1
889,42.4184,42.4912,42.4520,28416.098,213.420154,0.0728,183.333333,NaN,5.174342,Saline 2,rey,Animal 1
...,...,...,...,...,...,...,...,...,...,...,...,...
1547,3211.0936,3211.1648,3211.1344,23074.860,174.802633,0.0712,180.303030,NaN,5.397063,Saline 2,rey,Animal 1
1548,3460.8520,3460.9864,3460.8992,28050.031,240.996217,0.1344,153.030303,NaN,8.437627,Saline 2,rey,Animal 1
1549,3498.3248,3498.6480,3498.4184,459914.750,2710.669139,0.3232,134.848485,NaN,49.355525,Saline 2,rey,Animal 1
1550,3503.5424,3503.6032,3503.5712,135621.900,192.507190,0.0608,122.727273,NaN,8.367303,Saline 2,rey,Animal 1


In [15]:
df_count_ilknur = ilknur_new_df.loc[:, ["session", "animal", "start"]].groupby(["animal", "session"]).count()
df_count_ilknur.rename(columns={"start": "rpl_count"})

rpl_count
animal session              
finn   Saline 1          484
       Psilocybin        584
       Saline 2          372
finn2  Saline 1          739
       Psilocybin        782
       Saline 2          496
rey    Saline 1          418
       Psilocybin        467
       Saline 2          667
rose   Saline 1          546
       Psilocybin        499
       Saline 2          682

### Plot ripple features over time!

#### TODOs for this plot
1) Use only POST times, will require a bit of upfront work or post-hoc subtraction of the injection time.
2) Figure out why there is a peak for Saline1 at 2000 sec and Saline2 at the end - sleep?

In [ ]:
bin_size = 60

bins = np.arange(np.floor(df_grp.peak_time.min() / bin_size) * bin_size, df_grp.stop.max() + bin_size, bin_size).astype(int)
idb = pd.cut(df_grp.peak_time, bins, labels=False)
df_grp["Time bin (sec)"] = bins[idb]
df_grp

In [ ]:
%matplotlib widget
_, ax = plt.subplots()
feature = "duration"
sns.scatterplot(data=df_grp, x = "peak_time", y="duration", hue="Animal", size=1, ax=ax)

_, ax = plt.subplots()
sns.lineplot(data=df_grp, x="Time bin (sec)", y="duration", hue="session", ax=ax)

## Plotting template here with appropriate size graphics and parameters

In [ ]:
# Get data for POST sessions only and ONLY including 1 hr of POST
df_post1hr = df_grp[(df_grp.start > 0) & (df_grp.stop < 3600)]
df_post1hr

In [ ]:
features = ["duration", "peak_power", "peak_frequency_bp", "sharp_wave_amplitude"]
titles = ["Duration (s)", "Peak Power", "Peak Frequency", "SW Amplitude"]
fig, axs = plt.subplots(2, 2, figsize=(6, 4))
for idf, feature in enumerate(features):
    ax = axs.reshape(-1)[idf]
    plot_legend = True if idf == 0 else False
    sns.boxplot(data=df_post1hr, x="Animal", y=feature, dodge=True, hue="session", showfliers=False, fill=False, 
                legend=plot_legend, ax=ax)
    sns.stripplot(data=df_post1hr, x="Animal", y=feature, dodge=True, hue="session", size=2, 
                  linewidth=0.1, edgecolor="w", alpha=0.3, rasterized=rasterize_scatter,
                  legend=False, ax=ax)
    ax.set_ylabel(titles[idf])
    sns.despine(ax=ax)

fig.savefig(plot_dir / "ripple_features.pdf", dpi=600)

Alternative plotting method with each animal on different subplots

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(3, 2))
idf, feature = 0, "duration"
for ida, animal in enumerate([1, 2, 3, 4]):
    ax = axs.reshape(-1)[ida]
    plot_legend = True if ida == 0 else False
    df_animal = df_post1hr[df_post1hr.Animal == animal]
    sns.boxplot(data=df_animal, x="session", y=feature, hue="session", showfliers=False, fill=False, 
                linewidth=0.5, legend=plot_legend, ax=ax)
    sns.stripplot(data=df_animal, x="session", y=feature, hue="session", size=1, jitter=0.2,
                  linewidth=0.1, edgecolor="w", alpha=0.3, rasterized=rasterize_scatter,
                  legend=False, ax=ax)
    ax.set_ylabel(titles[idf])
    ax.set_xlabel(f"Animal {animal}")
    ax.set_xticklabels([])
    if ida == 0:
        sns.despine(ax=ax)
    else:
        sns.despine(ax=ax, left=True)
        ax.set_yticks([])
        ax.set_ylabel("")

fig.savefig(plot_dir / "ripple_features_alt.pdf", dpi=600)

### Check import of Ilknur's ripple data

In [ ]:
ilk_df = pd.read_csv("/Users/nkinsky/Documents/UM/Working/Psilocybin/Recording_Rats/SWR_all_animals_group_data.csv")
ilk_df["session"] = pd.Categorical(ilk_df["session"], ["Saline 1", "Psilocybin", "Saline 2"])
ilk_df

In [ ]:
ilk_df[(ilk_df.Animal_name == "Rose") & (ilk_df.session == "Saline 1")]

In [ ]:
df_post1hr[(df_post1hr.Animal_name == "Rose") & (df_post1hr.session == "Saline 1")]

In [ ]:
ilk_df.groupby(by=["Animal_name", "session"]).count()

In [ ]:
df_post1hr.groupby(by=["Animal_name", "session"]).count()